In [41]:
import cv2
import numpy as np
import symforce
import open3d as o3d
from numba import jit, njit
import numba as nb

symforce.set_symbolic_api("sympy")
symforce.set_log_level("warning")

from symforce.notebook_util import display
import symforce.symbolic as sf

colorImgs = []
depthImgs = []
poses = []

# Load images
for i in range(1,6):
    colorImgs.append(cv2.imread("./color/%s.png"%i))
    depthImgs.append(cv2.imread("./depth/%s.pgm"%i, -1))

# poses
print(len(colorImgs))
print(len(depthImgs))

5
5


In [45]:
def readTrajectory(path):

    trajectory = []

    with open(path) as f:
        for line in f:
            components = line.split(" ")
            tx, ty, tz, qx, qy, qz, qw = map(lambda x: float(x), components)
            rot = sf.Rot3(sf.Quaternion(sf.V3(qx, qy, qz), qw))
            trans = sf.V3(tx,ty,tz)
            p = sf.Pose3(R = rot, t= trans)
            trajectory.append(p)

    return trajectory

poses = readTrajectory("poses.txt")
print(len(poses))

5


In [46]:
@njit
def rangeToCloud(color, depth, T):
    cx = 325.5
    cy = 253.5
    fx = 518.0
    fy = 519.0
    depthScale = 1000.0
    pointcloud = nb.typed.List()
    colors = nb.typed.List()
    for v in np.arange(0, color.shape[0]):
        for u in np.arange(0, color.shape[1]):
            d = depth[v, u]/depthScale
            if d != 0:  # 0 means no valid value
                point = np.array([(u - cx)*d/fx, (v - cy)*d/fy, d, 1])
                pointWorld = np.dot(T, point)
                pointcloud.append(pointWorld[:3])
                colors.append(color[v][u]/255.0)
    return pointcloud, colors

In [47]:
Colors = []
Points = []
for i in range(5):
    print(f"Converting image {i+1}")
    color = colorImgs[i]
    depth = depthImgs[i]
    T = np.asarray(poses[i].to_homogenous_matrix().to_list()).astype(float)
    p, c = rangeToCloud(color, depth, T)
    Colors = Colors + np.array(c).tolist()
    Points = Points + np.array(p).tolist()

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(Points)
pcd.colors = o3d.utility.Vector3dVector(Colors)
o3d.visualization.draw_geometries([pcd])

Converting image 1
Converting image 2
Converting image 3
Converting image 4
Converting image 5
